## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-04-30-25 +0000,bbc,Sydney police to be investigated by watchdog a...,https://www.bbc.com/news/articles/crk84ggrv33o...
1,2026-02-13-04-23-11 +0000,nyt,What We Know About the Victims of the Tumbler ...,https://www.nytimes.com/2026/02/12/world/canad...
2,2026-02-13-04-22-21 +0000,nyt,Goldman Sachs General Counsel Kathryn Ruemmler...,https://www.nytimes.com/2026/02/12/business/go...
3,2026-02-13-04-19-46 +0000,nyt,"Trump Pardons Klecko, Jamal Lewis and Other Fo...",https://www.nytimes.com/2026/02/12/us/politics...
4,2026-02-13-04-18-00 +0000,wsj,Japan Seizes Chinese Fishing Boat and Arrests ...,https://www.wsj.com/world/asia/japan-seizes-ch...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2405/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,39
21,epstein,21
309,immigration,16
12,shooting,11
178,judge,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
140,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,135
201,2026-02-12-20-01-49 +0000,nyt,Trump Administration to End Surge of Immigrati...,https://www.nytimes.com/2026/02/12/us/minnesot...,88
203,2026-02-12-19-47-08 +0000,bbc,Minnesota immigration enforcement surge is end...,https://www.bbc.com/news/articles/c2lr9w29zwyo...,88
301,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...,86
42,2026-02-13-02-17-00 +0000,nypost,Minnesota Gov. Tim Walz slammed over proposed ...,https://nypost.com/2026/02/12/us-news/minnesot...,84


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
140,135,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
96,67,2026-02-12-23-36-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
13,66,2026-02-13-03-27-00 +0000,wsj,The Justice Department released what it says i...,https://www.wsj.com/us-news/law/what-we-know-s...
302,48,2026-02-12-14-41-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
301,47,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...
375,46,2026-02-12-08-39-36 +0000,wsj,"Stock Market News, Feb. 12, 2026: Tech Shares ...",https://www.wsj.com/livecoverage/stock-market-...
46,43,2026-02-13-02-11-46 +0000,nypost,Judge dismisses California deportation case fo...,https://nypost.com/2026/02/12/us-news/judge-di...
60,41,2026-02-13-01-39-02 +0000,startribune,DOJ moves to drop charges against men arrested...,https://www.startribune.com/doj-moves-to-drop-...
98,41,2026-02-12-23-33-00 +0000,wsj,When Customs and Border Protection fired a las...,https://www.wsj.com/politics/national-security...
94,37,2026-02-12-23-37-07 +0000,nyt,Seattle to Pay $29 Million to Family of Woman ...,https://www.nytimes.com/2026/02/12/us/seattle-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
